<a href="https://colab.research.google.com/github/eliasnetoDS/Breaking-Bad---Webscraping/blob/main/breaking_bad_webscraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Vamos acessar o link da página de avaliações da Série Breaking Bad no Site IMDB**

In [ ]:
from requests import get
url = 'https://www.imdb.com/title/tt0903747/episodes?season=1'    #aqui estamos acessando a primeira temporada da séire
response = get(url)
print(response.text[:250])

In [ ]:
from bs4 import BeautifulSoup                       #criação de um objeto beautifulsoup-não sei o que significa

html_soup = BeautifulSoup(response.text, 'html.parser')
type(html_soup)               

In [ ]:
episode_containers = html_soup.find_all('div', class_='info')

In [ ]:
episode_containers[0].a['title']        #título do episódio

'Pilot'

In [ ]:
episode_containers[0].meta['content']           #número do episódio

'1'

In [ ]:
episode_containers[0].find('div', class_='airdate').text.strip()   #ano do episódio no ar

'20 Jan. 2008'

In [ ]:
episode_containers[0].find('span', class_='ipl-rating-star__rating').text   #nota no imdb - classificação

'9.1'

In [ ]:
episode_containers[0].find('span', class_='ipl-rating-star__total-votes').text   #total de votos

'(25,865)'

In [ ]:
episode_containers[0].find('div', class_='item_description').text.strip()

'Diagnosed with terminal lung cancer, chemistry teacher Walter White teams up with former student Jesse Pinkman to cook and sell crystal meth.'

**VAMOS RODAR TODOS OS PASSOS ANTERIORES**

In [ ]:
# Initializing the series that the loop will populate
community_episodes = []

# Aqui o contador vai variar de 1 até a quantidade de temporadas + 1
for sn in range(1,6):
    # Request from the server the content of the web page by using get(), and store the server’s response in the variable response
    response = get('https://www.imdb.com/title/tt0903747/episodes?season=' + str(sn))

    # Parse the content of the request with BeautifulSoup
    page_html = BeautifulSoup(response.text, 'html.parser')

    # Select all the episode containers from the season's page
    episode_containers = page_html.find_all('div', class_ = 'info')

    # For each episode in each season
    for episodes in episode_containers:
            # Get the info of each episode on the page
            season = sn
            episode_number = episodes.meta['content']
            title = episodes.a['title']
            airdate = episodes.find('div', class_='airdate').text.strip()
            rating = episodes.find('span', class_='ipl-rating-star__rating').text
            total_votes = episodes.find('span', class_='ipl-rating-star__total-votes').text
            desc = episodes.find('div', class_='item_description').text.strip()
            # Compiling the episode info
            episode_data = [season, episode_number, title, airdate, rating, total_votes, desc]

            # Append the episode info to the complete dataset
            community_episodes.append(episode_data)




In [ ]:
#criando um dataframe

import pandas as pd 
community_episodes = pd.DataFrame(community_episodes, columns = ['season', 'episode_number', 'title', 'airdate', 'rating', 'total_votes', 'desc'])

community_episodes.head()

,season,episode_number,title,airdate,rating,total_votes,desc
0,1,1,Pilot,20 Jan. 2008,9.0,"(32,227)","Diagnosed with terminal lung cancer, chemistry..."
1,1,2,Cat's in the Bag...,27 Jan. 2008,8.6,"(23,576)",After their first drug deal goes terribly wron...
2,1,3,...And the Bag's in the River,10 Feb. 2008,8.7,"(22,871)",Walt and Jesse clean up after the bathtub inci...
3,1,4,Cancer Man,17 Feb. 2008,8.2,"(22,082)",Walt tells the rest of his family about his ca...
4,1,5,Gray Matter,24 Feb. 2008,8.3,"(21,685)",Walt rejects everyone who tries to help him wi...


**NESSA PRÓXIMA FASE VAMOS FAZER UMA LIMPEZA DOS DADOS**

**CONVERTER O TOTAL DE VOTOS QUE AGORA ESTÁ COMO TEXTO**

In [ ]:
def remove_str(votes):
    for r in ((',',''), ('(',''),(')','')):
        votes = votes.replace(*r)
        
    return votes


    community_episodes['total_votes'] = community_episodes.total_votes.apply(remove_str).astype(int)

community_episodes.head()

,season,episode_number,title,airdate,rating,total_votes,desc
0,1,1,Pilot,20 Jan. 2008,9.0,"(32,227)","Diagnosed with terminal lung cancer, chemistry..."
1,1,2,Cat's in the Bag...,27 Jan. 2008,8.6,"(23,576)",After their first drug deal goes terribly wron...
2,1,3,...And the Bag's in the River,10 Feb. 2008,8.7,"(22,871)",Walt and Jesse clean up after the bathtub inci...
3,1,4,Cancer Man,17 Feb. 2008,8.2,"(22,082)",Walt tells the rest of his family about his ca...
4,1,5,Gray Matter,24 Feb. 2008,8.3,"(21,685)",Walt rejects everyone who tries to help him wi...


**TRANSFORMAR EM NÚMEROS REAIS A AVALIAÇÃO QUE AGORA ESTÁ NO FORMATO DE TEXTO**

In [ ]:
community_episodes['rating'] = community_episodes.rating.astype(float)

**Convertendo a data de transmissão de string em data e hora**

In [ ]:
community_episodes['airdate'] = pd.to_datetime(community_episodes.airdate)

community_episodes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62 entries, 0 to 61
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   season          62 non-null     int64         
 1   episode_number  62 non-null     object        
 2   title           62 non-null     object        
 3   airdate         62 non-null     datetime64[ns]
 4   rating          62 non-null     object        
 5   total_votes     62 non-null     object        
 6   desc            62 non-null     object        
dtypes: datetime64[ns](1), int64(1), object(5)
memory usage: 3.5+ KB


**É HORA DE SALVAR O ARQUIVO**


In [ ]:
community_episodes.to_csv('breakingbad.csv',index=False)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
